In [1]:
%reset -f

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextGenerationPipeline, GPTQConfig, pipeline
from auto_gptq import exllama_set_max_input_length

import platform
import os
import shutil

import spacy

from math import ceil
from spacy.lang.en import English 

In [3]:
if platform.system() != 'Linux':
    
    print("GPTQ is optimized for NV cards on cuda-enabled Linux. Please check your OS")

else :
    
    os.chdir("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp")
    print("Linux detected.")
    print("the current wd is ", os.getcwd())


Linux detected.
the current wd is  /home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp


In [4]:
model_name_or_path = "/home/ning/workdir/model/wd-v0.0.1"
# model_name_or_path = "TheBloke/Llama-2-70B-GPTQ"
# To use a different branch, change revision
# For example: revision="main"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="gptq-4bit-128g-actorder_True")

# model = exllama_set_max_input_length(model, 2048)

# model.save_pretrained("/home/ning/workdir/model/docsum-testmodel-v0.0.1", from_pt=True) 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
start_wrd = "Summary:"

tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-GPTQ", use_fast=True)

# sliding windows : evaluating prompts $\geqslant$ 4096 tokens

In [9]:
with open("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_transformer_GPTQ/data/feature/test1.txt") as f:
    doc = f.read()

nlp = spacy.load("en_core_web_sm") 

result = nlp(doc)

window_size = 1000
grid_size = 2000
shards_number = ceil(len(result)/grid_size)
file_shards = [None] * shards_number

for i in range(shards_number):

    tmp = result[i *grid_size : i * grid_size + window_size]
    file_shards[i] = ''.join([token.text_with_ws for token in tmp])

    with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_transformer_GPTQ/data/feature/test1_' + str(i) + '.txt', 'w', encoding="utf-8") as f:
        f.write(file_shards[i])

In [11]:
short_sum = []

pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1500,
        do_sample=True,
        temperature=0.5,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.2
    )

for i in range(shards_number):

    with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_transformer_GPTQ/data/feature/test1_' + str(i) + '.txt', 'r+', encoding="utf-8") as file:
        context = file.read()



    prompt_template=f'''You are a helpful and patient AI assistant. You always pay attentions to all the details. Take a deep breathe and write a summary step by step about the following artile. Include all the details. \n
    Article: {short_sum} \n {context} \n 
    Summary: 
    '''

    result = pipe(prompt_template)

    raw_short_result = result[0]['generated_text']
    short_sum_idx = raw_short_result.find(start_wrd)
    short_sum = raw_short_result[short_sum_idx+8:]

with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_transformer_GPTQ/result/cumulated_sum.txt', 'w', encoding="utf-8") as f:
    f.write(short_sum)

print(short_sum)

 
    
Hami Qutami, who tried to arrange a murder, was given a harsher sentence increase of around 29 months by the court because he committed a severe crime and needed punishment that matched the harm done to society. The judges thought about many things when deciding the length of his punishment, including his help with police officers, his past, and his current health. Even though Qutami asked for a lighter sentence of about five years, the court decided that this punishment wouldn't fit the crime he committed.

The government argued that reducing the time before parole was wrong and wanted to keep the original long period on parole. However, Qutami believed his sentence was correct and balanced the seriousness of his actions and the special circumstances of his situation. He also stated that the court should not intervene without strong evidence, and that parts of his argument might be considered twice, causing unfairness.

Finally, the court decided that while Qutami's initial sen

# evaluating prompts $\leqslant$ 4096 tokens

In [8]:
with open("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_transformer_GPTQ/data/feature/os-1.txt") as f:
    context = f.read()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
    temperature=0.5,
    top_p=0.95,
    top_k=30,
    repetition_penalty=1.2
)

short_prompt_template=f'''You are a helpful and patient AI assistant. You always pay attentions to all the details. Take a deep breathe and write a long summary step by step about the following artile. Include all the details. \n
 Article: {context} \n 
 Summary: 
'''

result = pipe(short_prompt_template)

raw_short_result = result[0]['generated_text']
short_sum_idx = raw_short_result.find(start_wrd)
short_sum = raw_short_result[short_sum_idx+8:]
print(short_sum)

with open('/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_transformer_GPTQ/result/short_sum.txt', 'w', encoding="utf-8") as f:
    f.write(short_sum)

 
This article presents a detailed medical report regarding a 55-year-old male patient named Mr. Tan Ah Kow. The report includes essential patient's particulars like his full name, NRIC number, and current residential address. It highlights the doctor's credentials and their professional relationship. 

The document provides comprehensive insights into the patient's medical history, including previous health issues such as hypertension, hyperlipidemia, multiple strokes, cardiomyopathy, cardiac failure, and chronic renal disease. These complications led to numerous hospitalizations, transfers between hospitals, and treatments.

It further describes the patient's behavioral and psychological symptoms associated with Dementia, supported by findings from various physical and mental state examinations. These include observations of his mood, orientation to time, place, and person, responses to simple arithmetic tasks, understanding of financial matters, and knowledge of his medical conditio